In [1]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import datetime as dt
import alpaca_trade_api as tradeapi
import json
from alpaca_trade_api.rest import REST, TimeFrame
load_dotenv()

True

In [2]:
alpaca_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

tradeapi = REST(alpaca_key, alpaca_secret_key)

In [3]:
def get_daily_trade_data(ticker, start, end, tradeapi): 
    ticker_df = tradeapi.get_bars(
        ticker,
        TimeFrame.Day,
        start,
        end
    ).df
    ticker_df['ticker'] = ticker

    return ticker_df

In [4]:

# get_daily_trade_data('HD', '2022-01-03', '2022-01-07', tradeapi)

In [5]:
ticker_list = pd.read_csv('./Data/Cleaned_Data/Ticker_library.csv')['Ticker'].to_list()

def make_tickers_df(ticker_list, start_date_str, end_date_str, tradeapi):
    ticker_dfs_list = [get_daily_trade_data(ticker, start_date_str, end_date_str, tradeapi) for ticker in ticker_list]
    tickers_df = pd.concat(ticker_dfs_list, axis=0, join='outer')
    tickers_df.index = tickers_df.index.date
    tickers_df = tickers_df[['ticker','close','volume']]
    return tickers_df
    
    #['symbol','close'].reset_index().rename(columns={'timestamp':'date'}).set_index('date').dt.date
    #return tickers_df.index.dt.date.reset_index().rename(columns={"timestamp":"date"})

In [6]:
# stocks_df = make_tickers_df(ticker_list, '2012-06-01', '2022-06-01', tradeapi)
# stocks_df.head()

In [7]:
def make_tickers_dict(ticker_list, start_date_str, end_date_str, tradeapi):
    return {ticker: get_daily_trade_data(ticker, start_date_str, end_date_str, tradeapi) for ticker in ticker_list}

In [8]:
# ticker_list = ['AAPL', 'XOM']
# my_tick_dict = make_tickers_dict(ticker_list, '2022-01-03', '2022-01-07', tradeapi)
# my_tick_dict

## News Article Vader Analyzer

In [33]:
from libs.drew_lib import *

from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\altma\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [10]:
api_key = os.getenv("NEWSAPI_KEY")
newsapi = NewsApiClient(api_key=api_key)
print(type(api_key))

<class 'str'>


In [11]:
# company_df = pd.read_csv('./Data/Cleaned_Data/Ticker_library.csv')
# company_df = company_df.rename(columns={'Company Name': 'Company_Name'})
# company_dict = dict(zip(company_df.Company_Name, company_df.Ticker))
# company_list = list(company_dict.keys())
# tickers_list = company_dict.values()
# ls = [type(item) for item in company_list]
# print(ls)
# tickers_list


In [12]:
# netflix_news_df = form_df('Netflix')
# netflix_news_df

In [13]:
from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\altma\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [14]:
# def vader(df):
#     company_list = company_dict.keys()

#     df_list = []
#     for company in company_list:
#         try:
#             df_list.append(form_df(company))
#         except:
#             continue
#     news_df = pd.concat(df_list, axis=0, join='outer')
    
#     return news_df



In [15]:
def vader_analyzer(df):
    analyzer = SentimentIntensityAnalyzer()
    df['compound'] = [analyzer.polarity_scores(x)['compound'] for x in df['text']]
    df['neg'] = [analyzer.polarity_scores(x)['neg'] for x in df['text']]
    df['neu'] = [analyzer.polarity_scores(x)['neu'] for x in df['text']]
    df['pos'] = [analyzer.polarity_scores(x)['pos'] for x in df['text']]

    df['date'] = pd.to_datetime(
    df['date'],
    infer_datetime_format = True,
    utc = True    
    )
    df['date'] = df['date'].dt.date
    
    return df



In [16]:

# def daily_sentiment(df):
#     df = vader_analyzer(df)

#     df['date'] = pd.to_datetime(
#     df['date'],
#     infer_datetime_format = True,
#     utc = True    
#     )
    
#     return df.groupby('date')['compound','pos','neu','neg'].mean()


# # date to datetime

#     df['date'] = pd.to_datetime(
#     df['date'],
#     infer_datetime_format = True,
#     utc = True    
#     )
#     df['date'] = df['date'].dt.date


In [17]:
# df = vader_analyzer(netflix_news_df)
# df

In [40]:
news_df_list = [form_df(company) for company in company_list]
vader_news_df_list = [vader_analyzer(df) for df in news_df_list]
vader_news_df = pd.concat(vader_news_df_list, axis=0, join='inner')
news_sent_avg = vader_news_df.groupby(['ticker','date'])['ticker','pos','neg','neu','compound'].mean().reset_index()
news_sent_avg = news_sent_avg[['date','ticker','pos','neg','neu','compound']]


news_sent_avg.to_csv('../Notebooks/Data/Cleaned_Data/news_sentiment.csv')

C:\Users\altma\AppData\Local\Temp\ipykernel_16988\3887123157.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  news_sent_avg = vader_news_df.groupby(['ticker','date'])['ticker','pos','neg','neu','compound'].mean().reset_index()


In [31]:
news_sent_df = pd.read_csv('../Notebooks/Data/Cleaned_Data/news_sentiment.csv')
news_sent_df = news_sent_df[['date','ticker','pos','neg','neu','compound']]
#news_sent_df.set_index('ticker',inplace=True)
#news_sent_df
news_sent_avg = news_sent_df.groupby(['ticker','date'])['ticker','pos','neg','neu','compound'].mean().reset_index()
cols = ['date','ticker','pos','neg','neu','compound']
news_sent_avg.columns = cols
news_sent_avg
news_sent_df.to_csv('../Notebooks/Data/Cleaned_Data/news_sentiment.csv')

C:\Users\altma\AppData\Local\Temp\ipykernel_16988\4020423608.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  news_sent_avg = news_sent_df.groupby(['ticker','date'])['ticker','pos','neg','neu','compound'].mean().reset_index()


## Helper Functions for PuttingItTogether

In [ ]:
def stock_pick_df(ticker):

    stock_df_list = []

    for filename in os.listdir(file_path):
        if filename.endswith(".csv") and filename != 'Ticker_library.csv':
            csv_df = pd.read_csv(file_path +'/'+ filename, parse_dates=True, infer_datetime_format=True,index_col='date')
            csv_df = csv_df.loc[csv_df['ticker'] == ticker]
            csv_df.drop(columns='ticker',axis=1,inplace=True)
            stock_df_list.append(csv_df)
    all_ticker_data_df = pd.concat(stock_df_list,axis=1,join='inner')
    all_ticker_data_df.insert(0, 'ticker', ticker)
    
    return all_ticker_data_df


In [ ]:
file_path = '../Notebooks/Data/Cleaned_Data'
ntfx_all_df = stock_pick_df('NFLX')
ntfx_all_df

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [ ]:
stocks_df = pd.read_csv('../Notebooks/Data/Cleaned_Data/stock_data.csv')

def add_pct_change(dataframe):
    col_headers = {}
    for col in dataframe.columns:
        col_headers[col] = col.replace('close','pct_change')
    return pd.concat([dataframe,
    dataframe.pct_change().rename(columns=col_headers)
    ],axis='columns')


In [ ]:
stocks_pct_change = add_pct_change(stocks_df)
stocks_pct_change

TypeError: unsupported operand type(s) for /: 'str' and 'str'